In [ ]:
import sys
sys.path.append('../input/iterative/iterative-stratification-master/')
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold

import numpy as np
import pandas as pd

from sklearn.metrics import log_loss

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import regularizers
import tensorflow_addons as tfa

import random

In [ ]:
X_train = pd.read_csv('/kaggle/input/lish-moa/train_features.csv')
X_test = pd.read_csv('/kaggle/input/lish-moa/test_features.csv')
y_train = pd.read_csv('/kaggle/input/lish-moa/train_targets_scored.csv')
submission = pd.read_csv('/kaggle/input/lish-moa/sample_submission.csv')

In [ ]:
X_train.set_index('sig_id', inplace=True)
X_test.set_index('sig_id', inplace=True)
y_train.set_index('sig_id', inplace=True)

In [ ]:
X_train.cp_time = X_train.cp_time // 24
X_train.cp_dose = X_train.cp_dose.map({'D1': 0, 'D2': 1})
X_test.cp_time = X_test.cp_time // 24
X_test.cp_dose = X_test.cp_dose.map({'D1': 0, 'D2': 1})

In [ ]:
X_train_moa = X_train[X_train.cp_type != 'ctl_vehicle'].drop(columns=['cp_type'])
X_test_moa = X_test[X_test.cp_type != 'ctl_vehicle'].drop(columns=['cp_type'])

# Don't forget to keep only proper rows in y_train
y_train_moa = y_train.loc[X_train_moa.index]

In [ ]:
X_train_moa.head()

In [ ]:
def mean_predictions(models_dense, X):
    y_pred_dense = [model.predict(X) for model in models_dense]
    return np.mean(y_pred_dense, axis=0)

def macro_log_loss(y_true, y_pred):
    if len(y_true.shape) == 1:
        return log_loss(y_true, y_pred, labels=[0, 1]), [log_loss(y_true, y_pred, labels=[0, 1])] 
    y_pred = np.maximum(np.minimum(y_pred, [[1 - 1e-15] * y_pred.shape[1]] * y_pred.shape[0]), [[1e-15] * y_pred.shape[1]] * y_pred.shape[0])
    losses = [log_loss(y_true[:, i], y_pred[:, i], labels=[0, 1]) for i in range(y_true.shape[1])]
    return np.mean(losses), losses

In [ ]:
def create_model():
    model = tf.keras.Sequential([
    tf.keras.layers.Input(N_FEATURES),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.3),
    tfa.layers.WeightNormalization(tf.keras.layers.Dense(1024, activation="relu")),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.4),

    tfa.layers.WeightNormalization(tf.keras.layers.Dense(1024, activation="relu")),  
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.3),
    tfa.layers.WeightNormalization(tf.keras.layers.Dense(N_TARGETS, activation="sigmoid"))
    ])
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate = LR), loss='binary_crossentropy', metrics=["accuracy"])
    return model

In [ ]:
def create_dense_model(input_shape, output_shape):
    inputs = keras.Input(shape=(input_shape,), name='drug')
    x = layers.BatchNormalization()(inputs)
    x = layers.Dropout(0.3)(x)
    x = tfa.layers.WeightNormalization(layers.Dense(units=1024, activation='relu', name='dense_1'))(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.4)(x)
    x = tfa.layers.WeightNormalization(layers.Dense(units=1024, activation='relu', name='dense_2'))(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.3)(x)
    output = tfa.layers.WeightNormalization(layers.Dense(output_shape, activation='sigmoid', kernel_regularizer=keras.regularizers.l2(l2=1e-5), 
                                                         name='predictions'))(x)
    model = keras.Model(inputs=inputs, outputs=output)
    opt = tfa.optimizers.AdamW(weight_decay=1e-5, learning_rate=1e-2)
    model.compile(optimizer=opt, loss='binary_crossentropy')
    return model

In [ ]:
callbacks = [
    keras.callbacks.EarlyStopping(
        monitor='val_loss',
        min_delta=1e-5,
        patience=3,
        verbose=0),
    keras.callbacks.ReduceLROnPlateau(
        monitor="val_loss",
        factor=0.1,
        patience=2)
]

In [ ]:
x_train_dense = X_train_moa.to_numpy()
y_train_dense = y_train_moa.to_numpy()

kf = MultilabelStratifiedKFold(n_splits=5, shuffle=True)
mlls = []
models = []
i = 1
for train, test in kf.split(x_train_dense, y_train_dense):
    print(f'FOLD {i}.', end=' ')
    i = i + 1
    
    from numpy.random import seed
    seed(train[0])
    tf.random.set_seed(train[0])

    dense = create_dense_model(x_train_dense.shape[1], y_train_dense.shape[1])
    history_dense = dense.fit(x_train_dense[train], y_train_dense[train],
                     batch_size=64,
                     epochs=100,
                     callbacks=callbacks,
                     validation_data=(x_train_dense[test], y_train_dense[test]),
                     shuffle=True, verbose=0)
    mll, _ = macro_log_loss(y_train_dense[test], dense.predict(x_train_dense[test]))
    models.append(dense)
    mlls.append(mll)
    print('Dense macro log loss:', mll)

In [ ]:
print(np.mean(mlls))

In [ ]:
preds = mean_predictions(models, X_test_moa)

In [ ]:
indexes = dict(zip(X_test_moa.index, range(X_test_moa.shape[0])))

In [ ]:
for idx, row in submission.iterrows():
    sig_id = row.sig_id
    if sig_id in indexes.keys():
        submission.loc[idx] = [sig_id] + [np.maximum(np.minimum(pred, 1 - 1e-3), 1e-3) for pred in preds[indexes[sig_id]]]
    else:
        submission.loc[idx] = [sig_id] + [0] * preds.shape[1]

In [ ]:
submission.to_csv('submission.csv', index=False)